<a href="https://colab.research.google.com/github/mehnaj871/Machine-Learning-Project/blob/hello-world/Copy_of_text_news_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook

     |████████████████████████████████| 720 kB 5.5 MB/s 
     |████████████████████████████████| 189 kB 45.8 MB/s 
     |████████████████████████████████| 46 kB 2.4 MB/s 
     |████████████████████████████████| 1.2 MB 29.9 MB/s 
     |████████████████████████████████| 56 kB 3.0 MB/s 
     |████████████████████████████████| 51 kB 229 kB/s 


## Mounting Google drive 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing Libraries

In [3]:
from fastai.text.all import *
import matplotlib.image as immg
from PIL import Image
from fastai.vision.widgets import *

In [5]:
import warnings
warnings.filterwarnings("ignore")

### Downloading Data

In [6]:
path = untar_data('https://s3.amazonaws.com/fast-ai-nlp/ag_news_csv.tgz')

In [7]:
path.ls()

(#4) [Path('/root/.fastai/data/ag_news_csv/test.csv'),Path('/root/.fastai/data/ag_news_csv/train.csv'),Path('/root/.fastai/data/ag_news_csv/classes.txt'),Path('/root/.fastai/data/ag_news_csv/readme.txt')]

In [8]:
!cp '/root/.fastai/data/ag_news_csv/readme.txt' '/content'

In [9]:
df_trn = pd.read_csv(path/'train.csv',header=None,sep=',')
df_trn.columns = ['class_id','title','description']
df_ts = pd.read_csv(path/'test.csv',header=None,sep=',')
df_ts.columns = ['class_id','title','description']

In [10]:
df = pd.concat([df_trn,df_ts]).reset_index(drop=True)

In [11]:
df.head()

,class_id,title,description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again."
1,3,Carlyle Looks Toward Commercial Aerospace (Reuters),"Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market."
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
3,3,Iraq Halts Oil Exports from Main Southern Pipeline (Reuters),"Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday."
4,3,"Oil prices soar to all-time record, posing new menace to US economy (AFP)","AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections."


In [12]:
df.shape

(127600, 3)

### Image labels

## Analysing data

## Creating a data loader for training

In [13]:
dls_lm = TextDataLoaders.from_df(df.sample(frac=0.25),path='.',
                                 text_col=['title','description'],
                                 valid_pct=0.15,
                                 is_lm=True,bs=128)

In [14]:
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos xxfld 1 xxmaj sprint and xxmaj nextel to xxmaj merge , xxmaj creating xxmaj cellphone xxmaj powerhouse xxfld 2 xxmaj the deal creates the third - largest company in the wireless telephone industry , but its competition is much larger . xxbos xxfld 1 xxmaj spain fear xxmaj bryan brothers in xxmaj davis xxmaj cup final xxfld 2 xxmaj the xxmaj united xxmaj states doubles pairing of xxmaj mike and xxmaj","xxfld 1 xxmaj sprint and xxmaj nextel to xxmaj merge , xxmaj creating xxmaj cellphone xxmaj powerhouse xxfld 2 xxmaj the deal creates the third - largest company in the wireless telephone industry , but its competition is much larger . xxbos xxfld 1 xxmaj spain fear xxmaj bryan brothers in xxmaj davis xxmaj cup final xxfld 2 xxmaj the xxmaj united xxmaj states doubles pairing of xxmaj mike and xxmaj bob"
1,"stores lack excitement , executive says xxfld 2 xxup san xxup diego -- xxmaj music mogul xxmaj clive xxmaj davis has some advice for retailers looking to persuade music fans to return to traditional record shops : xxmaj make shopping more fun . xxbos xxfld 1 xxmaj delta xxmaj may xxmaj offer xxmaj its xxmaj pilots xxmaj equity xxmaj stake to xxmaj get xxmaj cuts xxfld 2 xxmaj delta xxmaj air xxmaj","lack excitement , executive says xxfld 2 xxup san xxup diego -- xxmaj music mogul xxmaj clive xxmaj davis has some advice for retailers looking to persuade music fans to return to traditional record shops : xxmaj make shopping more fun . xxbos xxfld 1 xxmaj delta xxmaj may xxmaj offer xxmaj its xxmaj pilots xxmaj equity xxmaj stake to xxmaj get xxmaj cuts xxfld 2 xxmaj delta xxmaj air xxmaj lines"
2,"elbaradei xxmaj says xxmaj unclear if xxmaj iran xxmaj nuke xxmaj plans xxmaj peaceful xxfld 2 xxup vienna ( reuters ) - xxmaj it is unclear if xxmaj iran 's nuclear ambitions are entirely peaceful , but there is still no firm evidence that xxmaj tehran is secretly developing atomic weapons as xxmaj washington asserts , the xxup u.n . nuclear watchdog said xxmaj tuesday . xxbos xxfld 1 xxmaj microsoft :","xxmaj says xxmaj unclear if xxmaj iran xxmaj nuke xxmaj plans xxmaj peaceful xxfld 2 xxup vienna ( reuters ) - xxmaj it is unclear if xxmaj iran 's nuclear ambitions are entirely peaceful , but there is still no firm evidence that xxmaj tehran is secretly developing atomic weapons as xxmaj washington asserts , the xxup u.n . nuclear watchdog said xxmaj tuesday . xxbos xxfld 1 xxmaj microsoft : xxmaj"
3,"by \ $ 101 . a … xxbos xxfld 1 xxmaj senate xxmaj foes xxmaj daschle , xxmaj xxunk xxmaj swaps xxmaj barbs in xxmaj debate ( reuters ) xxfld 2 xxmaj reuters - xxmaj senate xxmaj democratic leader xxmaj tom \ xxmaj daschle , fighting for political survival , angrily rejected \ xxmaj republic challenger xxmaj john xxmaj xxunk 's claim on xxmaj sunday that he had \ emboldened the enemy","\ $ 101 . a … xxbos xxfld 1 xxmaj senate xxmaj foes xxmaj daschle , xxmaj xxunk xxmaj swaps xxmaj barbs in xxmaj debate ( reuters ) xxfld 2 xxmaj reuters - xxmaj senate xxmaj democratic leader xxmaj tom \ xxmaj daschle , fighting for political survival , angrily rejected \ xxmaj republic challenger xxmaj john xxmaj xxunk 's claim on xxmaj sunday that he had \ emboldened the enemy in"
4,"inning gave xxmaj houston a 3 - 0 win over the xxmaj st xxmaj louis xxmaj cardinals xxmaj monday to move the xxmaj astros within one game of reaching the xxmaj world xxmaj series for the first time in their xxunk history . xxbos xxfld 1 palmsource buys into xxmaj china , xxmaj linux xxfld 2 com xxmaj december 8 , 2004 , xxunk xxup am xxup pt . palmsource , which","gave xxmaj houston a 3 - 0 win over the xxmaj st xxmaj louis xxmaj cardinals xxmaj monday to move the xxmaj astros within one game of reaching the xxmaj world xxmaj series for the first time in their xxunk history . xxbos xxfld 1 palmsource buys into xxmaj china , xxmaj linux xxfld 2 com xxmaj december 8 , 2004 , xxunk xxup am xxup pt . palmsource , which makes"


In [15]:
len(dls_lm.train_ds),len(dls_lm.valid_ds)

(27115, 4785)

### Training the Model

In [16]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [17]:
cb1 = SaveModelCallback(monitor='perplexity',fname='best_model',comp=np.less) # Callbacks
learn.fit_one_cycle(1, 1e-2, cbs =[cb1])

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.055243,3.527062,0.392179,34.023865,04:40


Better model found at epoch 0 with perplexity value: 34.02386474609375.


In [18]:
cb1 = SaveModelCallback(monitor='perplexity',fname='best_model',comp=np.less) # Callbacks
learn.fit_one_cycle(5, 1e-3, cbs =[cb1])

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.810388,3.493278,0.394682,32.893597,04:39
1,3.732790,3.432014,0.401856,30.938890,04:38
2,3.677222,3.399397,0.405628,29.946041,04:38
3,3.626684,3.385558,0.407507,29.534479,04:38
4,3.620296,3.383792,0.407485,29.482353,04:40


Better model found at epoch 0 with perplexity value: 32.89359664916992.
Better model found at epoch 1 with perplexity value: 30.93889045715332.
Better model found at epoch 2 with perplexity value: 29.946041107177734.
Better model found at epoch 3 with perplexity value: 29.53447914123535.
Better model found at epoch 4 with perplexity value: 29.48235321044922.


## Interpreting Results

In [19]:
learn = learn.to_fp32()
learn.export('/content/newsLM_export.pkl');

In [20]:
!cp '/content/newsLM_export.pkl' '/content/drive/MyDrive/'

## Inference

In [21]:
!pip install -qq gradio

     |████████████████████████████████| 979 kB 5.2 MB/s 
     |████████████████████████████████| 206 kB 46.4 MB/s 
     |████████████████████████████████| 2.0 MB 36.1 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 961 kB 41.1 MB/s 
     |████████████████████████████████| 3.6 MB 26.5 MB/s 


In [22]:
import gradio as gr

In [23]:
inf_learn = load_learner('/content/newsLM_export.pkl',cpu=False);

In [ ]:
def text_complete(txt,num,tmp):
    res = learn.predict(txt, n_words=int(num),temperature=tmp/100)
    return res

iface = gr.Interface(text_complete, 
                     [gr.inputs.Textbox(label='Text'),
                      gr.inputs.Slider(20, 100, default=30, label="num_words"),
                      gr.inputs.Slider(10, 100, default=95, label="temperature")], 
                      gr.outputs.Textbox());


if __name__ == "__main__":
    iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://27685.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app
